In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import json
import pickle
import time

In [3]:
result['data']['지정']

,CGG_CODE,ASGN_YY,ASGN_SNO,APPL_YMD,ASGN_YMD,UPSO_NM,SITE_ADDR_RD,SITE_ADDR,PERM_NT_NO,SNT_UPTAE_NM,MAIN_EDF,TRDP_AREA,ADMDNG_NM,GRADE_FACIL_GBN,UPSO_SITE_TELNO
0,3220000,2014,0160,20141006,20141006,이도곰탕,"서울특별시 강남구 논현로94길 29-5, 지상1층,지상2층 (역삼동)",서울특별시 강남구 역삼동 671번지 17호,3220000-101-2001-24197,한식,곰탕,194.41,역삼1동,상수도전용,02 5010738
1,3220000,2021,139,20181001,20211116,솥내음 스타필드 코엑스몰점,"서울특별시 강남구 영동대로 513, 코엑스 지하1층 O-107호 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,3220000-101-2020-00643,한식,직화불백,74.56,삼성1동,,
2,3220000,2010,0024,20100210,20100210,스시히로바,"서울특별시 강남구 삼성로 620, 블래스톤리조트 지상1층 (삼성동)",서울특별시 강남구 삼성동 70번지 블래스톤리조트,3220000-101-2002-00383,일식,초밥,216.56,삼성1동,상수도전용,02 5155511
3,3220000,2016,0089,20161125,20161125,서백자 간장게장,"서울특별시 강남구 삼성로 542, 지상2층 (삼성동, 석천빌딩)",서울특별시 강남구 삼성동 151번지 4호 석천빌딩 2층,3220000-101-2012-00416,중국식,간장게장,260.30,삼성1동,,02 34536008
4,3220000,2016,0084,20161125,20161125,주)봉산집,"서울특별시 강남구 삼성로 564, 지상2층,지상3층 (삼성동)",서울특별시 강남구 삼성동 145번지 19호,3220000-101-2011-01048,한식,차돌박이,452.95,삼성1동,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,3220000,2016,7,20161125,20161125,밀란,"서울특별시 강남구 개포로28길 4, (개포동,지상1층101호)",서울특별시 강남구 개포동 1215번지 0호 지상1층101호,3220000-101-2004-00840,한식,면류,128.76,개포4동,상수도전용,02 574 3216
312,3220000,2023,313,20231010,20231123,모스가든,"서울특별시 강남구 논현로139길 12, 지상1층 (논현동)",서울특별시 강남구 논현동 30번지 6호,3220000-101-2012-00814,한식,파스타,332.00,논현1동,,
313,3220000,2010,0058,20100210,20100210,삼성물산(주)10꼬르소꼬모서울카페,"서울특별시 강남구 압구정로 416, (청담동,지하1층)",서울특별시 강남구 청담동 79번지 0호 지하1층,3220000-101-2008-00093,경양식,이태리,350.00,청담동,상수도전용,02 30181010
314,3220000,2023,299,20231010,20231123,현대낙지집,"서울특별시 강남구 압구정로14길 11, (신사동)",서울특별시 강남구 신사동 550번지 3호,3220000-101-1985-17501,한식,"낙지, 감자탕",92.03,신사동,상수도전용,02 5461022


In [205]:
request_id = 14569639
url = f'https://place.map.kakao.com/main/v/{request_id}'
response = requests.get(url)
comments = []
if response.status_code == 200:
    comment_data = json.loads(response.content)['comment']
    comment_count = int(comment_data['kamapComntcnt'])
    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
    for c in comment_data['list']:
        comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
        request_comment_id = c['commentid']
    print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
else:
    print('error')

총 댓글 개수: 169개, 평점: 4.11점


In [206]:
while len(comments) < comment_count:
    url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
    response = requests.get(url)
    if response.status_code == 200:
        comment_data = json.loads(response.content)['comment']
        for c in comment_data['list']:
            try:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
            except:
                comments.append({'점수': c['point'], '댓글': '', '작성일': c['date']})
            request_comment_id = c['commentid']
    else:
        print('error')

In [207]:
pd.DataFrame(comments)

,점수,댓글,작성일
0,5,,2024.04.22.
1,2,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...,2024.04.14.
2,5,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...,2024.04.11.
3,5,수육이랑 곰탕먹었어요\n곰탕은 주문하자마자 빨리 나왔어요\n고기도 많이 들어가있고\...,2024.04.08.
4,5,세종대왕의 이름 (이도 李裪)에 걸맞는 \n진한 맛의 곰탕입니다\n마침 제옆에 외국...,2024.04.02.
...,...,...,...
164,4,곰탕,2016.09.29.
165,4,소수육46. 면+죽 추가10.,2016.09.09.
166,2,공장형.\n밥대신 국수.\n잘 모르겠다.,2016.08.03.
167,4,,2016.07.10.


[카카오맵 API]
1. 식당 id를 찾는다.

[카카오맵 크롤링]

2. 해당 식당 id를 url로 하여 requests를 보낸다.
3. 해당 response에서 댓글을 찾은 뒤 수집한다. (3개)
4. 마지막 댓글의 id를 찾아 새로운 url을 생성하여 requests를 보낸다.
5. 해당 response에서 댓글을 찾은 뒤 수집한다. (5개)
6. 모든 댓글을 수집할 때까지 4 - 5 단계를 반복한다.

---

In [4]:
df = result['data']['지정']

In [42]:
def find_string(txt):
    if '주)' in txt:
        target_index = txt.index('주)')
        if target_index > len(txt) // 2:
            return txt[:txt.index('주)') - 1]
        else:
            return txt[txt.index('주)') + 2:]
    else:
        return txt

df['UPSO_NM'].apply(find_string).str.split('(').str[0].str.rstrip('점')

0               이도곰탕
1      솥내음 스타필드 코엑스몰
2              스시히로바
3           서백자 간장게장
4                봉산집
           ...      
311               밀란
312             모스가든
313      10꼬르소꼬모서울카페
314            현대낙지집
315               초심
Name: UPSO_NM, Length: 316, dtype: object

In [152]:
key = ''
kakao_result = []
df['keyword'] = df['UPSO_NM'].apply(find_string).str.split('(').str[0].str.rstrip('점')
keywords = df['keyword'].tolist()

def generate_address(txt):
    try:
        if txt.split(' ')[5]:
            if txt.split(' ')[5][-1] == '호':
                return txt.split(' ')[2] + ' ' + txt.split(' ')[4].rstrip('번지') + '-' + txt.split(' ')[5].rstrip('호')
            else:
                return txt.split(' ')[2] + ' ' + txt.split(' ')[4].rstrip('번지')
        else:
            return txt.split(' ')[2] + ' ' + txt.split(' ')[4].rstrip('번지')
            
    except:
        return False
    
df = df.drop('id', axis=1)

In [242]:
kakao_result = []
rows = []
ids = []
for idx, keyword in enumerate(keywords[309:310]):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json' 
    params = {'query': keyword, 'page': 1}
    headers = {"Authorization": f'''KakaoAK {key}'''}
    response = requests.get(url, params=params, headers=headers)
    this_result = []
    this_id = []
    for row in json.loads(response.content)['documents']:
        rows.append(row)
        cond_gangnam = row['address_name'].split(' ')[0] == '서울' and row['address_name'].split(' ')[1] == '강남구'
        cond_addr = generate_address(df.loc[idx, 'SITE_ADDR']) == row['address_name'][7:]
        cond_road_addr = df.loc[idx, 'SITE_ADDR_RD'].replace('특별시', '').split(',')[0] == row['road_address_name']
        if cond_gangnam and (cond_addr or cond_road_addr):
            this_result.append({
                '검색어': keyword,
                '상호명': row['place_name'],
                '지번주소': row['address_name'],
                '도로명주소': row['road_address_name'],
                'id': row['id']
            })
            this_id.append(row['id'])
    ids.append(this_id if this_id else None)
#     if len(this_result) == 1:
#         df.loc[idx, 'id'] = this_result[0]['id']
# kakao_df = pd.DataFrame(kakao_result)

In [232]:
df

,CGG_CODE,ASGN_YY,ASGN_SNO,APPL_YMD,ASGN_YMD,UPSO_NM,SITE_ADDR_RD,SITE_ADDR,PERM_NT_NO,SNT_UPTAE_NM,MAIN_EDF,TRDP_AREA,ADMDNG_NM,GRADE_FACIL_GBN,UPSO_SITE_TELNO,keyword,id
0,3220000,2014,0160,20141006,20141006,이도곰탕,"서울특별시 강남구 논현로94길 29-5, 지상1층,지상2층 (역삼동)",서울특별시 강남구 역삼동 671번지 17호,3220000-101-2001-24197,한식,곰탕,194.41,역삼1동,상수도전용,02 5010738,이도곰탕,[14569639]
1,3220000,2021,139,20181001,20211116,솥내음 스타필드 코엑스몰점,"서울특별시 강남구 영동대로 513, 코엑스 지하1층 O-107호 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,3220000-101-2020-00643,한식,직화불백,74.56,삼성1동,,,솥내음 스타필드 코엑스몰,[710342577]
2,3220000,2010,0024,20100210,20100210,스시히로바,"서울특별시 강남구 삼성로 620, 블래스톤리조트 지상1층 (삼성동)",서울특별시 강남구 삼성동 70번지 블래스톤리조트,3220000-101-2002-00383,일식,초밥,216.56,삼성1동,상수도전용,02 5155511,스시히로바,"[7974994, 1794437422]"
3,3220000,2016,0089,20161125,20161125,서백자 간장게장,"서울특별시 강남구 삼성로 542, 지상2층 (삼성동, 석천빌딩)",서울특별시 강남구 삼성동 151번지 4호 석천빌딩 2층,3220000-101-2012-00416,중국식,간장게장,260.30,삼성1동,,02 34536008,서백자 간장게장,[24663994]
4,3220000,2016,0084,20161125,20161125,주)봉산집,"서울특별시 강남구 삼성로 564, 지상2층,지상3층 (삼성동)",서울특별시 강남구 삼성동 145번지 19호,3220000-101-2011-01048,한식,차돌박이,452.95,삼성1동,,,봉산집,"[15105092, 2088113318]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,3220000,2016,7,20161125,20161125,밀란,"서울특별시 강남구 개포로28길 4, (개포동,지상1층101호)",서울특별시 강남구 개포동 1215번지 0호 지상1층101호,3220000-101-2004-00840,한식,면류,128.76,개포4동,상수도전용,02 574 3216,밀란,"[9011542, 25702443]"
312,3220000,2023,313,20231010,20231123,모스가든,"서울특별시 강남구 논현로139길 12, 지상1층 (논현동)",서울특별시 강남구 논현동 30번지 6호,3220000-101-2012-00814,한식,파스타,332.00,논현1동,,,모스가든,[540731064]
313,3220000,2010,0058,20100210,20100210,삼성물산(주)10꼬르소꼬모서울카페,"서울특별시 강남구 압구정로 416, (청담동,지하1층)",서울특별시 강남구 청담동 79번지 0호 지하1층,3220000-101-2008-00093,경양식,이태리,350.00,청담동,상수도전용,02 30181010,10꼬르소꼬모서울카페,[22400767]
314,3220000,2023,299,20231010,20231123,현대낙지집,"서울특별시 강남구 압구정로14길 11, (신사동)",서울특별시 강남구 신사동 550번지 3호,3220000-101-1985-17501,한식,"낙지, 감자탕",92.03,신사동,상수도전용,02 5461022,현대낙지집,[8148270]


In [171]:
check_df = pd.merge(
    left=df,
    right=kakao_df.groupby('검색어',as_index=False).agg({'id': 'count'}),
    left_on='keyword',
    right_on='검색어',
    how='left'
)[['UPSO_NM', 'id']].fillna(0)

check_df[check_df['id'] == 0]

,UPSO_NM,id
5,더드림,0.0
6,국민한우집 삼성역점(구 칠프로칠백식당 포스코점),0.0
9,등대,0.0
10,(주)아구본가첨벙대치,0.0
13,경성면옥,0.0
...,...,...
297,로즈힐 강남파이낸스센터점,0.0
303,한쉐프,0.0
304,지강한식당 압구정본점,0.0
309,효미역 압구정점,0.0


---

In [2]:
review_df = pd.read_csv('reviews.csv')

In [5]:
by_store = review_df.groupby('stoe_name').agg('count')
by_store

,user,rate,avg_rate,review,review_count
stoe_name,,,,,
(주)곰바위,15,16,16,9,16
(주)닭으로가,64,64,64,54,64
(주)도라후구현복집,15,16,16,9,16
(주)신정,2,2,2,2,2
(주)아구본가첨벙대치,20,20,20,9,20
...,...,...,...,...,...
홍영재 장수 청국장,30,30,30,19,30
화기애애(강남1호점),47,47,47,39,47
황금돼지,6,6,6,5,6


- ~같은 식당에 **중복 유저** 있는 경우 있음 ... 이걸 어떻게 처리할지?~
    - ~재방문 손님(해당 식당에 만족한 손님)이 있다는 뜻이므로 가중치를 준다~
    - ~특정 유저의 편향성에 weight가 들어갈 수 있으므로 중복을 제거한다~
    - ~일단은 그대로 두고 진행한다~
- 리뷰 개수가 **너무 적은 식당**은 어떻게 해야 할지?
    - 특정한 성향에 편중된 결과가 도출될 가능성 있으니 해당 식당은 제외한다
    - 처리할 방도가 없을 것 같으니 일단은 그대로 두고 진행한다
- **댓글 내용이 없는 경우**는 어떻게 해야 할지?
    - 댓글 내용이 없는 경우 무성의한 댓글일 수 있으니 제거하고 진행한다
    - 댓글 내용은 없더라도 별점은 있으니 그대로 두고 진행한다

### 중복 유저

In [6]:
by_store[by_store['user'] != by_store['rate']]

,user,rate,avg_rate,review,review_count
stoe_name,,,,,
(주)곰바위,15,16,16,9,16
(주)도라후구현복집,15,16,16,9,16
(주)우동명가기리야마본진,135,136,136,109,136
구이가,44,46,46,43,46
니뽕내뽕(코엑스점),53,54,54,50,54
만두의 전설,29,30,30,24,30
미사리밀빛초계국수,78,79,79,67,79
배정철 어도,45,46,46,21,46
송화,28,29,29,23,29


---

In [6]:
df = pd.read_csv('request_reviews.csv', index_col=0)
small_df = df[['BPLCNM', 'exemplary_diner', 'id_mod2']]
small_df.head()

,BPLCNM,exemplary_diner,id_mod2
Unnamed: 0,,,
0,배초향,NaN,
1,브래서리,모범식당지정,10825881
2,로비라운지,NaN,727661240
3,로비바,NaN,21550576
4,노블발렌티,NaN,17157328


In [7]:
small_df['id_mod2_p'] = small_df.loc[:, 'id_mod2'].apply(lambda x: np.nan if x == ' ' else x)
small_df

C:\Users\louis\AppData\Local\Temp\ipykernel_24408\2249939075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['id_mod2_p'] = small_df.loc[:, 'id_mod2'].apply(lambda x: np.nan if x == ' ' else x)


,BPLCNM,exemplary_diner,id_mod2,id_mod2_p
Unnamed: 0,,,,
0,배초향,NaN,,NaN
1,브래서리,모범식당지정,10825881,10825881
2,로비라운지,NaN,727661240,727661240
3,로비바,NaN,21550576,21550576
4,노블발렌티,NaN,17157328,17157328
...,...,...,...,...
12523,EPIK 에픽,NaN,,NaN
12524,(주)탭샵바,NaN,,NaN
12525,밥한민국,NaN,,NaN


In [8]:
small_df['id_mod2_p']

Unnamed: 0
0              NaN
1         10825881
2        727661240
3         21550576
4         17157328
           ...    
12523          NaN
12524          NaN
12525          NaN
12526          NaN
12527          NaN
Name: id_mod2_p, Length: 12528, dtype: object

In [12]:
small_df.loc[12000:]

,BPLCNM,exemplary_diner,id_mod2,id_mod2_p
Unnamed: 0,,,,
12000,식당 민홍,NaN,,NaN
12001,바로푸드,NaN,,NaN
12002,곰프레시,NaN,,NaN
12003,곰푸드,NaN,,NaN
12004,분식을 품다,NaN,,NaN
...,...,...,...,...
12523,EPIK 에픽,NaN,,NaN
12524,(주)탭샵바,NaN,,NaN
12525,밥한민국,NaN,,NaN


In [13]:
small_df.isna().sum()

BPLCNM                 0
exemplary_diner    12212
id_mod2                9
id_mod2_p           6072
dtype: int64

In [9]:
for i, request_id in enumerate(small_df.loc[4176:4178]['id_mod2_p'], 4176):
    print(i, request_id)
    url = f'https://place.map.kakao.com/main/v/{request_id}'
    print(url)

4176 nan
https://place.map.kakao.com/main/v/nan
4177 17058166
https://place.map.kakao.com/main/v/17058166
4178 21447066
https://place.map.kakao.com/main/v/21447066


In [19]:
import time

In [42]:
comments = []
for i, request_id in enumerate(small_df.loc[4176:8352]['id_mod2_p'], 4176):
    time.sleep(1)
    if request_id == np.nan:
        continue
    else:
        url = f'https://place.map.kakao.com/main/v/{request_id}'
        response = requests.get(url)
        
        if response.status_code == 200:
            try:
                comment_data = json.loads(response.content)['comment']
                comment_count = int(comment_data['kamapComntcnt'])
                if comment_count == 0:
                    continue
                overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
                for c in comment_data['list']:
                    comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                        'review': c['contents'], 'date': c['date'],
                                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    request_comment_id = c['commentid']
            
                while len(comments) < comment_count:
                    url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
                    response = requests.get(url)
                    if response.status_code == 200:
                        comment_data = json.loads(response.content)['comment']
                        for c in comment_data['list']:
                            try:
                                comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                            'review': c['contents'], 'date': c['date'],
                                                            'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                            "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                            except:
                                comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                            'review': "", 'date': c['date'],
                                                            'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                            "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                            request_comment_id = c['commentid']
                    else:
                        print('error')
            except:
                continue
        else:
            print('error')

        

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


KeyboardInterrupt: 

In [ ]:
request_id = 14569639
url = f'https://place.map.kakao.com/main/v/{request_id}'
response = requests.get(url)
comments = []
if response.status_code == 200:
    comment_data = json.loads(response.content)['comment']
    comment_count = int(comment_data['kamapComntcnt'])
    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
    for c in comment_data['list']:
        comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
        request_comment_id = c['commentid']
    print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
else:
    print('error')

while len(comments) < comment_count:
    url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
    response = requests.get(url)
    if response.status_code == 200:
        comment_data = json.loads(response.content)['comment']
        for c in comment_data['list']:
            try:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
            except:
                comments.append({'점수': c['point'], '댓글': '', '작성일': c['date']})
            request_comment_id = c['commentid']
    else:
        print('error')

---

In [13]:
df = pd.read_csv('request_reviews_240428.csv', index_col=0)
df

,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
Unnamed: 0,,,,,,,,,,,,,,,,,,,
0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,203520.881091,446950.285626,NaN,NaN,청담동,NaN,배초향,NaN,,
1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881
2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240
3,3220000-101-1899-20463,19991006,영업/정상,282.59,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비바,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비바,['21550576'],21550576,21550576
4,3220000-101-1899-20500,19991012,영업/정상,"1,157.90",135873,서울특별시 강남구 삼성동 109-6,서울 강남구 봉은사로 637,노블발렌티,한식,205662.873886,445913.719788,결혼예식장주변,상수도전용,삼성동,NaN,노블발렌티,"['17157328', '26597965']",17157328,17157328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12523,3220000-101-2024-00510,2024-04-23,영업/정상,81.00,135-812,서울특별시 강남구 논현동 27 노블빌딩,서울 강남구 논현로145길 41,EPIK 에픽,한식,202223.582536,446044.245465,NaN,NaN,논현동,NaN,EPIK 에픽,NaN,,
12524,3220000-101-2024-00511,2024-04-23,영업/정상,33.05,135-814,서울특별시 강남구 논현동 23-10,서울 강남구 강남대로136길 23,(주)탭샵바,경양식,201909.686539,445710.536195,NaN,NaN,논현동,NaN,탭샵바,NaN,,
12525,3220000-101-2024-00512,2024-04-23,영업/정상,140.49,135-825,서울특별시 강남구 논현동 148-13,서울 강남구 강남대로124길 68,밥한민국,한식,202335.109233,445416.506981,NaN,NaN,논현동,NaN,밥한민국,NaN,,


In [14]:
small_df = df[['BPLCNM', 'exemplary_diner', 'id_mod2']]
small_df

,BPLCNM,exemplary_diner,id_mod2
Unnamed: 0,,,
0,배초향,NaN,
1,브래서리,모범식당지정,10825881
2,로비라운지,NaN,727661240
3,로비바,NaN,21550576
4,노블발렌티,NaN,17157328
...,...,...,...
12523,EPIK 에픽,NaN,
12524,(주)탭샵바,NaN,
12525,밥한민국,NaN,


In [15]:
small_df['id_mod2_p'] = small_df.loc[:, 'id_mod2'].apply(lambda x: np.nan if x == ' ' else x)
small_df

C:\Users\louis\AppData\Local\Temp\ipykernel_5948\2249939075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['id_mod2_p'] = small_df.loc[:, 'id_mod2'].apply(lambda x: np.nan if x == ' ' else x)


,BPLCNM,exemplary_diner,id_mod2,id_mod2_p
Unnamed: 0,,,,
0,배초향,NaN,,NaN
1,브래서리,모범식당지정,10825881,10825881
2,로비라운지,NaN,727661240,727661240
3,로비바,NaN,21550576,21550576
4,노블발렌티,NaN,17157328,17157328
...,...,...,...,...
12523,EPIK 에픽,NaN,,NaN
12524,(주)탭샵바,NaN,,NaN
12525,밥한민국,NaN,,NaN


In [42]:
comments = []
start_idx = 4176
end_idx = 8352
# end_idx = 4185
data_size = end_idx - start_idx + 1
for i, request_id in enumerate(small_df.loc[start_idx:end_idx]['id_mod2_p'], start_idx):
    if ((i - start_idx) * 100 / data_size) % 10 == 0:
        print(f'''{'*' * ((i - start_idx) * 10 // data_size)}{'-' * (10 - ((i - start_idx) * 10 // data_size))} [{(i - start_idx) * 100 / data_size}%] | 현재 식당 번호: {i}, 수집 댓글 개수: {len(comments)}''')
        time.sleep(3)
    comment_count = 0
    
    if request_id == np.nan:
        continue
    
    else:
#         time.sleep(1)
        url = f'https://place.map.kakao.com/main/v/{request_id}'
        response = requests.get(url)
        
        if response.status_code == 200:
            try:
                comment_data = json.loads(response.content)['comment']
                comment_count = int(comment_data['kamapComntcnt'])
                now_comment_count = min(3, comment_count)
                
                if comment_count == 0:
                    continue
                    
                overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
                for c in comment_data['list']:
                    comments.append({"name": small_df.loc[i, "BPLCNM"], "user": c["username"], 
                                                        'review': c['contents'], 'date': c['date'],
                                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    request_comment_id = c['commentid']
            
                while now_comment_count < comment_count:
                    url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
                    response = requests.get(url)
                    if response.status_code == 200:
                        comment_data = json.loads(response.content)['comment']
                        for c in comment_data['list']:
                            try:
                                user = c["username"]
                            except:
                                user = ''
                                
                            try:
                                review = c['contents']
                            except:
                                review = ''
                            
                            try:
                                comments.append({"name": small_df.loc[i, "BPLCNM"], "user": user, 
                                                            'review': review, 'date': c['date'],
                                                            'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                            "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                            except:
                                comments.append({"name": small_df.loc[i, "BPLCNM"], "user": np.nan,
                                                 'review': np.nan, 'date': np.nan, 'rate': np.nan,
                                                 'avg_rate':  np.nan, 'review_cnt': np.nan, 'is_model': np.nan})
                                
                            request_comment_id = c['commentid']
                            now_comment_count += 1
                    else:
                        continue
            except:
                continue
        else:
            continue

---------- [0.0%] | 현재 식당 번호: 4176, 수집 댓글 개수: 0


In [45]:
pd.DataFrame(comments).to_csv('comments_4176_8352.csv')

In [19]:
df = pd.read_csv('comments_4176_8352.csv', index_col=0)
df

,name,user,review,date,rate,avg_rate,review_cnt,is_model
0,북창동 순두부 강남구청,uxxuey,고등어 맛있는데.. 두번먹었는데 안비렸음\n평일 점심 피크시간 피하면 좋음,2024.03.10.,5,3.7,42,NaN
1,북창동 순두부 강남구청,🍤,"고등어 비림, 목살구이는 그저 그럼\n불친절함",2023.12.20.,1,4.2,50,NaN
2,북창동 순두부 강남구청,다다,1인손님 안받으면 웨이팅 할 때 미리 말해줘야 하는거 아닌가요?\n기다렸다가 들어가...,2023.10.23.,1,2.3,3,NaN
3,북창동 순두부 강남구청점,나타리,차별은 없어서 좋네 1인 손님이고 단체손님이고 똑같이 불친절함 ㅋㅋㅋ,2023.10.07.,1,3.6,85,NaN
4,북창동 순두부 강남구청점,ㅇㅇㅇ,너뭌ㅋㅋㅋㅋㅋ 맛 없음&hellip;.\n\n일단 엄청 짜고 조미료맛만 남\n기본으...,2023.05.23.,1,3.1,39,NaN
...,...,...,...,...,...,...,...,...
49953,롯데시네마 도곡,s,알바들이 평점관리 하는듯 \n뭐 물어보면 말투 띠껍구 사람 하나도 없는데 키오스크 ...,2020.10.31.,1,2.2,9,NaN
49954,롯데시네마 도곡,YoungMo.K,이 지점은 매점 관리는 하나도 안하는듯..\n언제 만들었는지 모를 비닐주머니에 팝콘...,2020.08.13.,1,2.3,4,NaN
49955,롯데시네마 도곡,달콩이,씨네싸롱의 리클라이닝 좌석은 너무 좋은데 화장실 하수구냄새라고 해야하나... 냄새가...,2020.06.26.,3,3.6,26,NaN
49956,롯데시네마 도곡,전상진 장기렌트/리스,매봉역에 영화관이라니 굿굿,2020.06.06.,5,5.0,1,NaN


In [20]:
def find_string(txt):
    if '주)' in txt:
        target_index = txt.index('주)')
        if target_index > len(txt) // 2:
            return txt[:txt.index('주)') - 1]
        else:
            return txt[txt.index('주)') + 2:]
    else:
        return txt

# find_string 함수 적용 + "~점"으로 끝나는 가게 이름에서 "점" 제거
df["new_name"] = df['name'].apply(find_string).str.split('(').str[0].str.rstrip('점')

In [23]:
df[['new_name', 'user', 'review', 'date', 'rate', 'avg_rate', 'review_cnt', 'is_model']]

,new_name,user,review,date,rate,avg_rate,review_cnt,is_model
0,북창동 순두부 강남구청,uxxuey,고등어 맛있는데.. 두번먹었는데 안비렸음\n평일 점심 피크시간 피하면 좋음,2024.03.10.,5,3.7,42,NaN
1,북창동 순두부 강남구청,🍤,"고등어 비림, 목살구이는 그저 그럼\n불친절함",2023.12.20.,1,4.2,50,NaN
2,북창동 순두부 강남구청,다다,1인손님 안받으면 웨이팅 할 때 미리 말해줘야 하는거 아닌가요?\n기다렸다가 들어가...,2023.10.23.,1,2.3,3,NaN
3,북창동 순두부 강남구청,나타리,차별은 없어서 좋네 1인 손님이고 단체손님이고 똑같이 불친절함 ㅋㅋㅋ,2023.10.07.,1,3.6,85,NaN
4,북창동 순두부 강남구청,ㅇㅇㅇ,너뭌ㅋㅋㅋㅋㅋ 맛 없음&hellip;.\n\n일단 엄청 짜고 조미료맛만 남\n기본으...,2023.05.23.,1,3.1,39,NaN
...,...,...,...,...,...,...,...,...
49953,롯데시네마 도곡,s,알바들이 평점관리 하는듯 \n뭐 물어보면 말투 띠껍구 사람 하나도 없는데 키오스크 ...,2020.10.31.,1,2.2,9,NaN
49954,롯데시네마 도곡,YoungMo.K,이 지점은 매점 관리는 하나도 안하는듯..\n언제 만들었는지 모를 비닐주머니에 팝콘...,2020.08.13.,1,2.3,4,NaN
49955,롯데시네마 도곡,달콩이,씨네싸롱의 리클라이닝 좌석은 너무 좋은데 화장실 하수구냄새라고 해야하나... 냄새가...,2020.06.26.,3,3.6,26,NaN
49956,롯데시네마 도곡,전상진 장기렌트/리스,매봉역에 영화관이라니 굿굿,2020.06.06.,5,5.0,1,NaN


In [35]:
df['is_model_b'] = df['is_model'].map({np.nan: 0, '모범식당지정': 1})

In [42]:
df[['new_name', 'user', 'review', 'date', 'rate', 'avg_rate', 'review_cnt', 'is_model_b']].rename(columns={
    'new_name': 'name'
})

,name,user,review,date,rate,avg_rate,review_cnt,is_model_b
0,북창동 순두부 강남구청,uxxuey,고등어 맛있는데.. 두번먹었는데 안비렸음\n평일 점심 피크시간 피하면 좋음,2024.03.10.,5,3.7,42,0
1,북창동 순두부 강남구청,🍤,"고등어 비림, 목살구이는 그저 그럼\n불친절함",2023.12.20.,1,4.2,50,0
2,북창동 순두부 강남구청,다다,1인손님 안받으면 웨이팅 할 때 미리 말해줘야 하는거 아닌가요?\n기다렸다가 들어가...,2023.10.23.,1,2.3,3,0
3,북창동 순두부 강남구청,나타리,차별은 없어서 좋네 1인 손님이고 단체손님이고 똑같이 불친절함 ㅋㅋㅋ,2023.10.07.,1,3.6,85,0
4,북창동 순두부 강남구청,ㅇㅇㅇ,너뭌ㅋㅋㅋㅋㅋ 맛 없음&hellip;.\n\n일단 엄청 짜고 조미료맛만 남\n기본으...,2023.05.23.,1,3.1,39,0
...,...,...,...,...,...,...,...,...
49953,롯데시네마 도곡,s,알바들이 평점관리 하는듯 \n뭐 물어보면 말투 띠껍구 사람 하나도 없는데 키오스크 ...,2020.10.31.,1,2.2,9,0
49954,롯데시네마 도곡,YoungMo.K,이 지점은 매점 관리는 하나도 안하는듯..\n언제 만들었는지 모를 비닐주머니에 팝콘...,2020.08.13.,1,2.3,4,0
49955,롯데시네마 도곡,달콩이,씨네싸롱의 리클라이닝 좌석은 너무 좋은데 화장실 하수구냄새라고 해야하나... 냄새가...,2020.06.26.,3,3.6,26,0
49956,롯데시네마 도곡,전상진 장기렌트/리스,매봉역에 영화관이라니 굿굿,2020.06.06.,5,5.0,1,0


In [46]:
df.groupby('name').agg({'review': 'count'}).sort_values('review', ascending=False)

,review
name,
스시마이우 강남CGV점,890
비놀릭,583
(주)중앙해장,427
크라이치즈버거 강남점,386
더블유 앤 엠(W&M),363
...,...
유캔두잇 강남논현,0
탑에스,0
마포생소금구이,0


In [50]:
df.to_csv('comments_wj(4176-8352).csv')